**INTRODUCTION:**

The script elucidated below is designed for monitoring vegetation changes over time in the Sundarbans Mangroves, a significant region spanning across Bangladesh and India. Recognized as the largest mangrove forest globally, the Sundarbans are a UNESCO World Heritage Site and the abode of the renowned Royal Bengal Tiger. This region encapsulates a complex ecosystem comprising mangrove forests, swamps, and tidal waterways. Monitoring the Normalized Difference Vegetation Index (NDVI) in this area is imperative due to its ecological significance and the looming threats from climate change, sea-level rise, and human encroachment.

The data employed in this project is retrieved from the MODIS (Moderate Resolution Imaging Spectroradiometer) satellite's MOD13A2 product, courtesy of Kamel Didan - University of Arizona, Alfredo Huete - University of Technology Sydney and MODAPS SIPS - NASA (2015). This product furnishes NDVI values at a 16-day interval, aggregated annually in this analysis to discern the vegetation health and changes over the years in the Sundarbans. The specific dataset being used is MODIS/006/MOD13A2 (https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13A2)

The data processing and analysis entail several crucial steps including data extraction, aggregation, and visualization. This script effectively extracts MODIS NDVI data within the stipulated study period, aggregates the 16-day interval NDVI data to compute the mean NDVI value for each year, and visualizes the mean NDVI values on a map to delineate the spatial distribution of vegetation health in the Sundarbans. Furthermore, it employs Python libraries and tools like Matplotlib to visualize the processed data and fabricates a time-lapse video to vividly exhibit the change in NDVI values over time.

**INSTALL AND LOAD PACKAGES**  

Installation of rasterio and importing required libraries for the script execution.


In [ ]:
!pip install rasterio

In [ ]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import imageio
import os
from IPython.display import Image, display
from osgeo import gdal
import rasterio

**AUTHENTICATE EARTH ENGINE**

Authentication and initialization of Google Earth Engine to access geospatial datasets.

In [ ]:
# Initialize the Earth Engine module
ee.Authenticate()
ee.Initialize()

**CALCULATE YEARLY NDVI VALUES**

Defining the region of interest (Sundarbans Mangroves).
Creating a function (get_modis_ndvi) to fetch MODIS NDVI data for a specific year.
Iterating through the years to calculate the mean NDVI for each year, exporting the results to TIFF images.

In [ ]:
# Define the region of interest: Sundarbans Mangroves
sundarbans = ee.Geometry.Rectangle([88.0, 21.5, 90.0, 22.5])

# Function to get MODIS NDVI data for a specific year
def get_modis_ndvi(year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"

    # Use MOD13A2.006 product for NDVI
    dataset = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(start_date, end_date).filterBounds(sundarbans)

    # Extract NDVI and scale it. MODIS NDVI values are scaled by a factor of 0.0001
    ndvi = dataset.select('NDVI').mean().multiply(0.0001)

    return ndvi

# Export each yearly average NDVI image
image_list = []
for year in range(2000, 2024):
    ndvi_image = get_modis_ndvi(year)
    out_path = f'/content/NDVI_{year}.tif'
    geemap.ee_export_image(ndvi_image, filename=out_path, scale=500, region=sundarbans, file_per_band=False)
    image_list.append(out_path)

**DISPLAY A FEW OF THE PLOTS**  

Visualizing a subset of the generated NDVI images for selected years using Matplotlib.

In [ ]:

# Visualize a few of the downloaded images
for year in [2000, 2005, 2010, 2015, 2020]:
    img_path = f'/content/NDVI_{year}.tif'
    ds = gdal.Open(img_path)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray()
    plt.imshow(arr, cmap='BuGn', vmin=-0.1, vmax=0.9)
    plt.title(f'NDVI {year}')
    plt.colorbar()
    plt.show()

**CREATING THE GIF**

Defining a function to normalize array values.
Iterating through each year to read the NDVI images, normalize and apply a custom colormap, and add a year label to each image.
Creating a GIF from the sequence of images to visualize the time-lapse of NDVI values.

In [ ]:
# Directory where the images are saved
image_dir = '/content/'

# List to store the images
images = []

# Function to normalize the array values to 0-1 for visualization
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

# Define a custom colormap (red-yellow-green)
colors = ["blue", "green"]
cmap = plt.cm.colors.LinearSegmentedColormap.from_list("", colors)

# Loop through each year and read the images
for year in range(2000, 2024):
    image_path = os.path.join(image_dir, f'NDVI_{year}.tif')

    # Open the image using rasterio
    with rasterio.open(image_path) as src:
        array = src.read(1)

    # Normalize the array values
    norm_array = normalize(array)

    # Apply the colormap
    colored_array = (cmap(norm_array) * 255).astype(np.uint8)

    # Convert the colored array to an image
    img = Image.fromarray(colored_array)

    # Convert to RGB (to ensure compatibility with imageio)
    img_rgb = img.convert('RGB')

    # Add the year label to the image
    draw = ImageDraw.Draw(img_rgb)
    font = ImageFont.load_default()
    draw.text((20, 20), str(year), font=font, fill="white")

    # Append to the images list
    images.append(img_rgb)

# Create the GIF
out_gif_path = '/content/NDVI_timelapse.gif'
images[0].save(out_gif_path, save_all=True, append_images=images[1:], loop=0, duration=1000)

**LABELING GIF**

Defining a function to normalize array values.
Iterating through each year to read the NDVI images, normalize and apply a custom colormap, and add a year label to each image.
Creating a GIF from the sequence of images to visualize the time-lapse of NDVI values.

In [ ]:
# Directory where the images are saved
image_dir = '/content/'

# List to store the images
images = []

# Function to normalize the array values to 0-1 for visualization
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)


for year in range(2000, 2024):
    img_path = f"/content/NDVI_{year}.tif"
    img = rasterio.open(img_path).read(1)

    # Normalize the image array between 0 and 1 for visualization
    img_norm = (img - img.min()) / (img.max() - img.min())

    # Convert the normalized image array to an RGB image using the GnBu colormap
    img_rgb = (255 * plt.cm.GnBu_r(img_norm)).astype(np.uint8)

    # Convert the RGB image to a PIL Image
    img_rgb = Image.fromarray(img_rgb[:, :, :3])

    # Add the year label to the image
    draw = ImageDraw.Draw(img_rgb)
    font = ImageFont.load_default()
    draw.text((20, 20), str(year), font=font, fill="black")

    # Append the labeled image to the images list
    images.append(img_rgb)

# Create the GIF
out_gif_path = '/content/NDVI_timelapse_2.gif'
images[0].save(out_gif_path, save_all=True, append_images=images[1:], loop=0, duration=1000)

In summary, this script provides a systematic approach to calculate and visualize the trend of vegetation change over the Sundarbans Mangroves region across a span of 24 years. Through a series of well-structured steps, the script fetches MODIS NDVI data, processes it to calculate yearly mean NDVI, visualizes a subset of the yearly data, and ultimately creates a GIF to illustrate the temporal changes in vegetation. The resulting GIF offers an engaging way to understand the dynamics of vegetation in the region over the years.  

DATA CITATION:
MODIS/006/MOD13A2:
Kamel Didan - University of Arizona, Alfredo Huete - University of Technology Sydney and MODAPS SIPS - NASA. (2015). MOD13A2 MODIS/Terra Vegetation Indices 16-Day L3 Global 1km SIN Grid. NASA LP DAAC. http://doi.org/10.5067/MODIS/MOD13A2.006​1​.